In [1]:
import pandas as pd
import pickle
import segyio
from math import floor
import numpy as np
import matplotlib.pyplot as plt
from seysmo.models.model_class import Wav2Vec2Framework, Wav2vec2Loss, ConvFeatureExtractor, ContextualTransformerEncoder
from seysmo.models.train_model import train, evaluate
from seysmo.models.utils import save_model, load_model, count_parameters, EarlyStopper, SignalSpeedDataset, give_data
from seysmo.visualization.plotting import plot_map
from seysmo.features.mapping import do_array_for_mapping, compute_y_pred
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import MeanAbsolutePercentageError
%load_ext autoreload
%autoreload 2
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")

In [2]:
class ExampleFeatureExtractor(nn.Module):
    def __init__(self, extracted_feature_size):
        super().__init__()
        self.linear = nn.Linear(27, extracted_feature_size)

    def forward(self, inputs):
        hidden_states = self.linear(inputs)

        return hidden_states


class ExampleEncoder(nn.Module):
    def __init__(self, extracted_feature_size, encoder_hidden_size):
        super().__init__()
        self.linear = nn.Linear(extracted_feature_size, encoder_hidden_size)

    def forward(self, hidden_states):
        hidden_states = self.linear(hidden_states)

        return hidden_states

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = 'cpu'
print(f"Use device: {device}")

extracted_feature_size = 512
encoder_hidden_size = extracted_feature_size

feature_extractor = ConvFeatureExtractor().to(device)
encoder = ContextualTransformerEncoder(embed_dim=512, num_heads=8, num_layers=12).to(device)

# `(batch size, time steps, feature size)`
inputs = torch.randn(32, 313, 27).to(device)

code_vector_size =  512
num_code_vector_groups = 2
num_code_vectors_per_group = 100
mask_time_prob = 0.02
num_mask_time_steps = 3
gumbel_init_temperature = 2
contrastive_loss_temperature = 0.1
num_contrastive_loss_negative_samples = 10
loss_alpha = 0.1

model = Wav2Vec2Framework(mask_time_prob, num_mask_time_steps, num_code_vector_groups, num_code_vectors_per_group,
                 extracted_feature_size, code_vector_size, gumbel_init_temperature, encoder_hidden_size, feature_extractor, encoder).to(device)
criterion = Wav2vec2Loss(contrastive_loss_temperature, num_contrastive_loss_negative_samples, num_code_vector_groups,
                 num_code_vectors_per_group, loss_alpha)

# tuple(Encoder hidden states with mask, Quantized features, Code book perplexity, Time mask indices)
model_out = model(inputs)
loss = criterion(*model_out)

print(loss)
loss.backward()

Use device: cpu
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
tensor(2.3468, grad_fn=<AddBackward0>)


In [7]:
model_out[0].shape

torch.Size([32, 31, 512])

In [26]:
model_out[1].shape

torch.Size([4, 313, 512])

In [27]:
model_out[2].shape

torch.Size([2, 100])

In [28]:
model_out[3].shape

torch.Size([4, 313])

In [12]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Use device: {device}")

extracted_feature_size = 562
encoder_hidden_size = 1024

feature_extractor = ExampleFeatureExtractor(extracted_feature_size).to(device)
encoder = ExampleEncoder(extracted_feature_size, encoder_hidden_size).to(device)

# `(batch size, time steps, feature size)`
inputs = torch.randn(4, 313, 27).to(device)
# `(batch size)` Number of available time steps per batch
input_lengths = torch.tensor([1000, 871, 389, 487]).to(device)


model = Wav2Vec2Framework(mask_time_prob, num_mask_time_steps, num_code_vector_groups, num_code_vectors_per_group,
                 extracted_feature_size, code_vector_size, gumbel_init_temperature, encoder_hidden_size, feature_extractor, encoder).to(device)
criterion = Wav2vec2Loss(contrastive_loss_temperature, num_contrastive_loss_negative_samples, num_code_vector_groups,
                 num_code_vectors_per_group, loss_alpha)

# tuple(Encoder hidden states with mask, Quantized features, Code book perplexity, Time mask indices)
model_out = model(inputs)
loss = criterion(*model_out)

print(loss)

Use device: cuda:0


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
